# Funções "Windowa" (Window Functions) em PySpark

As Funções de Janela permitem realizar cálculos sobre um conjunto de linhas relacionadas à linha atual, sem precisar agrupar os dados como em GROUP BY.

**Conceitos importantes:**
- **PARTITION BY**: divide os dados em grupos (partições)
- **ORDER BY**: define a ordem dentro de cada partição
- **Frame**: define o conjunto de linhas para o cálculo (padrão: todas as linhas da partição)

In [1]:
# Instalar o OpenJDK 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar o Apache Spark (versão 3.5.0 com Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# Extrair o arquivo baixado
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Instalar a biblioteca findspark
!pip install -q findspark

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

# Criar SparkSession
spark = SparkSession.builder.appName('FuncoesJanela').getOrCreate()

In [3]:
# Dados de exemplo
dados_funcionarios = [
    ("João", "Vendas", 15000),
    ("Maria", "TI", 20000),
    ("Pedro", "Vendas", 15000),
    ("Andrey", "Vendas", 11000),
    ("Luan", "Vendas", 12000),
    ("Bruna", "Vendas", 13000),
    ("Ana", "Gerência", 25000),
    ("Paulo", "TI", 18000),
    ("Lucas", "Vendas", 10000),
    ("Carla", "TI", 22000),
    ("Roberto", "Gerência", 23000)
]
df = spark.createDataFrame(dados_funcionarios, ["nome", "departamento", "salario"])
df.show()

+-------+------------+-------+
|   nome|departamento|salario|
+-------+------------+-------+
|   João|      Vendas|  15000|
|  Maria|          TI|  20000|
|  Pedro|      Vendas|  15000|
| Andrey|      Vendas|  11000|
|   Luan|      Vendas|  12000|
|  Bruna|      Vendas|  13000|
|    Ana|    Gerência|  25000|
|  Paulo|          TI|  18000|
|  Lucas|      Vendas|  10000|
|  Carla|          TI|  22000|
|Roberto|    Gerência|  23000|
+-------+------------+-------+



### Window
O Window é um módulo do PySpark que permite criar janelas de análise, ou seja, conjuntos de linhas relacionadas à linha atual.
Ele não faz nenhum cálculo sozinho — ele só define a “janela” que será usada pelas funções como rank(), row_number(), lag(), etc.

**não cria um DataFrame nem calcula nada** — ela **define uma janela lógica**, que é usada pelas funções de janela (`rank()`, `row_number()`, `lag()`, etc.).

Ou seja:

* Você **não verá nada** apenas executando essa linha, porque não há dados sendo transformados ainda.
* Para visualizar o efeito da janela, você precisa **aplicar uma função de janela sobre ela**, como fizemos com `rank()`:

In [7]:
janela = Window.partitionBy("departamento").orderBy("salario")

In [8]:
# RANK
df_com_rank = df.withColumn("classificacao", F.rank().over(janela))
df_com_rank.orderBy("departamento", "salario").show()

+-------+------------+-------+-------------+
|   nome|departamento|salario|classificacao|
+-------+------------+-------+-------------+
|Roberto|    Gerência|  23000|            1|
|    Ana|    Gerência|  25000|            2|
|  Paulo|          TI|  18000|            1|
|  Maria|          TI|  20000|            2|
|  Carla|          TI|  22000|            3|
|  Lucas|      Vendas|  10000|            1|
| Andrey|      Vendas|  11000|            2|
|   Luan|      Vendas|  12000|            3|
|  Bruna|      Vendas|  13000|            4|
|   João|      Vendas|  15000|            5|
|  Pedro|      Vendas|  15000|            5|
+-------+------------+-------+-------------+



##  `F.dense_rank().over(janela)`

* `dense_rank()` é **uma variante do `rank()`**.
* Ela também dá uma **classificação dentro da partição**, considerando a ordem da coluna definida (`salario`).

**Diferença principal para `rank()`**:

* `rank()` pula posições quando há empates (lacunas).
* `dense_rank()` **não pula posições** — a próxima posição continua sequencial.

> Observe: com `rank()`, a próxima posição após o empate de João e Pedro seria 7 (lacuna).
> Com `dense_rank()`, continua 6 (sem lacunas).




In [9]:
# DENSE_RANK
df_com_rank = df_com_rank.withColumn("classificacao_densa", F.dense_rank().over(janela))
df_com_rank.orderBy("departamento", "salario").show()

+-------+------------+-------+-------------+-------------------+
|   nome|departamento|salario|classificacao|classificacao_densa|
+-------+------------+-------+-------------+-------------------+
|Roberto|    Gerência|  23000|            1|                  1|
|    Ana|    Gerência|  25000|            2|                  2|
|  Paulo|          TI|  18000|            1|                  1|
|  Maria|          TI|  20000|            2|                  2|
|  Carla|          TI|  22000|            3|                  3|
|  Lucas|      Vendas|  10000|            1|                  1|
| Andrey|      Vendas|  11000|            2|                  2|
|   Luan|      Vendas|  12000|            3|                  3|
|  Bruna|      Vendas|  13000|            4|                  4|
|   João|      Vendas|  15000|            5|                  5|
|  Pedro|      Vendas|  15000|            5|                  5|
+-------+------------+-------+-------------+-------------------+



###`percent_rank()`


`percent_rank()` **mostra a posição relativa de uma linha dentro do grupo**, em **percentual**, entre 0 e 1.

* 0 significa que a linha está **no início** da partição.
* 1 significa que a linha está **no final** da partição.


**Fórmula básica:**

$$
\text{percent_rank} = \frac{\text{rank} - 1}{\text{total de linhas na partição} - 1}
$$

* `rank` = classificação da linha (igual ao `rank()` que vimos antes)
* `total de linhas na partição` = número de linhas no grupo



In [10]:
# PERCENT_RANK
df_com_rank = df_com_rank.withColumn("percentil", F.percent_rank().over(janela))
df_com_rank.orderBy("departamento", "salario").show()

+-------+------------+-------+-------------+-------------------+---------+
|   nome|departamento|salario|classificacao|classificacao_densa|percentil|
+-------+------------+-------+-------------+-------------------+---------+
|Roberto|    Gerência|  23000|            1|                  1|      0.0|
|    Ana|    Gerência|  25000|            2|                  2|      1.0|
|  Paulo|          TI|  18000|            1|                  1|      0.0|
|  Maria|          TI|  20000|            2|                  2|      0.5|
|  Carla|          TI|  22000|            3|                  3|      1.0|
|  Lucas|      Vendas|  10000|            1|                  1|      0.0|
| Andrey|      Vendas|  11000|            2|                  2|      0.2|
|   Luan|      Vendas|  12000|            3|                  3|      0.4|
|  Bruna|      Vendas|  13000|            4|                  4|      0.6|
|   João|      Vendas|  15000|            5|                  5|      0.8|
|  Pedro|      Vendas|  1

### **`ROW_NUMBER()`**

* Atribui **um número sequencial único** a cada linha dentro da partição definida pela janela.
* Sempre **começa em 1** e aumenta de 1 em 1.
* **Não há empates**: mesmo que duas linhas tenham o mesmo valor, cada uma recebe um número diferente.




In [11]:
# ROW_NUMBER
df_com_rank = df_com_rank.withColumn("numero_linha", F.row_number().over(janela))
df_com_rank.orderBy("departamento", "salario").show()

+-------+------------+-------+-------------+-------------------+---------+------------+
|   nome|departamento|salario|classificacao|classificacao_densa|percentil|numero_linha|
+-------+------------+-------+-------------+-------------------+---------+------------+
|Roberto|    Gerência|  23000|            1|                  1|      0.0|           1|
|    Ana|    Gerência|  25000|            2|                  2|      1.0|           2|
|  Paulo|          TI|  18000|            1|                  1|      0.0|           1|
|  Maria|          TI|  20000|            2|                  2|      0.5|           2|
|  Carla|          TI|  22000|            3|                  3|      1.0|           3|
|  Lucas|      Vendas|  10000|            1|                  1|      0.0|           1|
| Andrey|      Vendas|  11000|            2|                  2|      0.2|           2|
|   Luan|      Vendas|  12000|            3|                  3|      0.4|           3|
|  Bruna|      Vendas|  13000|  

Perfeito! Vamos explicar de forma simples o **`LAG()`**.

---

### **`LAG()`**

* Retorna o **valor de uma coluna de uma linha anterior** dentro da mesma janela (partição).
* Sintaxe básica:

```python
F.lag(coluna, offset, default).over(janela)
```

* `coluna` → coluna que você quer “olhar para trás”
* `offset` → quantas linhas antes você quer olhar (padrão = 1)
* `default` → valor caso não exista linha anterior (opcional)

---

**Supondo o departamento Vendas, ordenado por salário:**

| nome   | salario | nome\_anterior | salario\_anterior |
| ------ | ------- | -------------- | ----------------- |
| Lucas  | 10000   | null           | null              |
| Andrey | 11000   | Lucas          | 10000             |
| Luan   | 12000   | Andrey         | 11000             |
| Bruna  | 13000   | Luan           | 12000             |
| João   | 15000   | Bruna          | 13000             |
| Pedro  | 15000   | João           | 15000             |

---

### Observações:

1. **Linha inicial da partição** → não tem valor anterior → retorna `null` (ou valor default, se definido).
2. `offset` maior que 1 → olha mais linhas para trás. Ex.: `lag("salario", 2)` retorna o valor de duas linhas antes.
3. Muito útil para **comparar valores com a linha anterior**, calcular diferenças ou crescimento entre linhas.

In [12]:
# LAG
df_com_navegacao = df.withColumn("nome_anterior", F.lag("nome", 1).over(janela))
df_com_navegacao = df_com_navegacao.withColumn("salario_anterior", F.lag("salario", 1).over(janela))
df_com_navegacao.orderBy("departamento", "salario").show()

+-------+------------+-------+-------------+----------------+
|   nome|departamento|salario|nome_anterior|salario_anterior|
+-------+------------+-------+-------------+----------------+
|Roberto|    Gerência|  23000|         NULL|            NULL|
|    Ana|    Gerência|  25000|      Roberto|           23000|
|  Paulo|          TI|  18000|         NULL|            NULL|
|  Maria|          TI|  20000|        Paulo|           18000|
|  Carla|          TI|  22000|        Maria|           20000|
|  Lucas|      Vendas|  10000|         NULL|            NULL|
| Andrey|      Vendas|  11000|        Lucas|           10000|
|   Luan|      Vendas|  12000|       Andrey|           11000|
|  Bruna|      Vendas|  13000|         Luan|           12000|
|   João|      Vendas|  15000|        Bruna|           13000|
|  Pedro|      Vendas|  15000|         João|           15000|
+-------+------------+-------+-------------+----------------+



### **`LEAD()`**

* É **o inverso do `LAG()`**.
* Retorna o **valor de uma coluna de uma linha seguinte** dentro da mesma janela (partição).
* Sintaxe básica:

```python
F.lead(coluna, offset, default).over(janela)
```

* `coluna` → coluna que você quer “olhar para frente”
* `offset` → quantas linhas à frente você quer olhar (padrão = 1)
* `default` → valor caso não exista linha seguinte (opcional)

---

**Supondo o departamento Vendas, ordenado por salário:**

| nome   | salario | nome\_proximo | salario\_proximo |
| ------ | ------- | ------------- | ---------------- |
| Lucas  | 10000   | Andrey        | 11000            |
| Andrey | 11000   | Luan          | 12000            |
| Luan   | 12000   | Bruna         | 13000            |
| Bruna  | 13000   | João          | 15000            |
| João   | 15000   | Pedro         | 15000            |
| Pedro  | 15000   | null          | null             |

---

### Observações

1. **Última linha da partição** → não tem linha seguinte → retorna `null` (ou valor default, se definido).
2. `offset` maior que 1 → olha mais linhas à frente. Ex.: `lead("salario", 2)` retorna o valor de duas linhas depois.
3. Muito útil para **comparar valores com a próxima linha**, calcular crescimento futuro ou prever tendências.

In [13]:
# LEAD
df_com_navegacao = df_com_navegacao.withColumn("nome_proximo", F.lead("nome", 1).over(janela))
df_com_navegacao = df_com_navegacao.withColumn("salario_proximo", F.lead("salario", 1).over(janela))
df_com_navegacao.orderBy("departamento", "salario").show()

+-------+------------+-------+-------------+----------------+------------+---------------+
|   nome|departamento|salario|nome_anterior|salario_anterior|nome_proximo|salario_proximo|
+-------+------------+-------+-------------+----------------+------------+---------------+
|Roberto|    Gerência|  23000|         NULL|            NULL|         Ana|          25000|
|    Ana|    Gerência|  25000|      Roberto|           23000|        NULL|           NULL|
|  Paulo|          TI|  18000|         NULL|            NULL|       Maria|          20000|
|  Maria|          TI|  20000|        Paulo|           18000|       Carla|          22000|
|  Carla|          TI|  22000|        Maria|           20000|        NULL|           NULL|
|  Lucas|      Vendas|  10000|         NULL|            NULL|      Andrey|          11000|
| Andrey|      Vendas|  11000|        Lucas|           10000|        Luan|          12000|
|   Luan|      Vendas|  12000|       Andrey|           11000|       Bruna|          13000|

In [16]:
analise_completa = (
    df
    .withColumn("posicao_salario", F.rank().over(janela))
    .withColumn("percentil_salario", F.percent_rank().over(janela))
    .withColumn("diferenca_anterior", F.col("salario") - F.lag("salario", 1).over(janela))
    .withColumn("maior_salario_depto", F.max("salario").over(Window.partitionBy("departamento")))
    .withColumn("menor_salario_depto", F.min("salario").over(Window.partitionBy("departamento")))
    .withColumn("media_salario_depto", F.avg("salario").over(Window.partitionBy("departamento")))
    .withColumn("diferenca_da_media", F.col("salario") - F.avg("salario").over(Window.partitionBy("departamento")))
)

analise_completa.orderBy("departamento", "salario").show(truncate=False)


+-------+------------+-------+---------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+
|nome   |departamento|salario|posicao_salario|percentil_salario|diferenca_anterior|maior_salario_depto|menor_salario_depto|media_salario_depto|diferenca_da_media|
+-------+------------+-------+---------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+
|Roberto|Gerência    |23000  |1              |0.0              |NULL              |25000              |23000              |24000.0            |-1000.0           |
|Ana    |Gerência    |25000  |2              |1.0              |2000              |25000              |23000              |24000.0            |1000.0            |
|Paulo  |TI          |18000  |1              |0.0              |NULL              |22000              |18000              |20000.0            |-2000.0           |
|Maria  |TI          |

**Dicas importantes sobre funções de janela:**
1. Sempre definir **ORDER BY** quando usar funções como rank(), row_number()
2. **PARTITION BY** é opcional - sem ele, considera todo o dataset como uma partição
3. Funções de janela **não reduzem o número de linhas** (diferente de GROUP BY)
4. Cuidado com performance em datasets muito grandes
5. Use frames personalizados para cálculos específicos (médias móveis, etc.)

**Casos de uso comuns:**
- Ranking de vendedores por região
- Top N produtos por categoria
- Crescimento comparativo período anterior
- Percentis de performance
- Médias móveis para análise temporal
- Identificação de outliers em grupos

In [ ]:
spark.stop()